# **Realimentação de estados**

A principal técnica de controle em espaço de estados é o que chamamos de "realimentação de estados".

A estratégia básica é fazer a lei de controle $u$ ser proporcional ao vetor de estados. 
$$
\begin{align*}
    u = -\mathbf{Kx}
\end{align*}
$$

Lembra um pouco o controle proporcional, realimentação negativa e o controlador é apenas uma constante. Mas a diferença básica é que, como o $\mathbf{x}$ é um vetor, o controlador $\mathbf{K}$ tem que ser um vetor também. 


No caso, o sinal de controle é um escalar. Sendo $\mathbf{x}$ um vetor coluna de $n$ elementos, então $\mathbf{K}$ deve ser um vetor linha de $n$ elementos. Assim:
$$
\begin{align*}
    \mathbf{K}= \left[\begin{array}{cccc}K_1 & K_2 & \ldots & K_n\end{array}\right]
\end{align*}
$$

O objetivo inicial da realimentação de estados é fazer com que os estados do sistema caminhem para zero (ponto de equilíbrio) a partir de um estado inicial $\mathbf{x}(0)$ diferente de zero, em regime permanente (isto é, passado um longo tempo). Esse problema é chamado de **regulação**.

Quando o controlador é capaz de fazer isso, tarefas um pouco mais complexas (tais como rastreamento) tornam-se possíveis. 


**Observação importante**: se prestar atenção, o controlador de estados é muito simples. É basicamente multiplicar cada estado por uma constante e somar o resultado antes de realimentar! Porém, ele tem uma limitação fundamental: a regulação supõe que **todos** os estados são conhecidos para calcular a ação de controle. 

Isso quase sempre não é verdade. Os estados são grandezas internas que, às vezes, nem conseguimos medir. Mas mesmo quando é possível, pode não ser economicamente viável comprar sensor para medir todo mundo. 

Na prática, o único sinal que está disponível, por definição, é o sinal de saída. 

Desta forma, o que precisamos fazer é construir um sistema auxiliar que forneça umas *estimativa* dos estados reais a partir do sinal de saída. Esse sistema é conhecido como estimador ou observador de estados. 

<p align="center">
<img src="Fig7.11.svg" width="70%">
</p>

O projeto e análise do observador de estados será visto mais a frente.

**Por enquanto vamos considerar que é possível medir todos os estados do sistema para fazer a realimentação**

## Análise da realimentação de estados
Para um sistema
$$
\begin{align*}
    \mathbf{\dot{x}} &= \mathbf{Fx+G}u
\end{align*}
$$
se usarmos a lei de controle $u = -\mathbf{Kx}$, a equação deixará de ter entrada, podendo ser resolvida a partir da condição inicial.

$$
\begin{align*}
    \mathbf{\dot{x}} &= \mathbf{Fx+G(-Kx)}\\
    &= \mathbf{Fx-GKx}\\
    &= \mathbf{(F-GK)x}\\
    \mathbf{\dot{x}} &= \mathbf{F}_m \mathbf{x}
\end{align*}
$$

Vamos chamar matriz $\mathbf{F}_m$ de "matriz de malha fechada". 

A última equação é uma EDO que não possui entrada, então só tem sentido resolvê-la para uma condição inicial $\mathbf{x}(0)$ diferente de zero.

A condição necessária para que os estados do sistema caminhem para zero em regime permanente é apenas que o sistema em malha fechada seja estável, isto é, **todos** os autovalores da matriz $\mathbf{F}_m$ devem ter parte real estritamente negativa. 

Além de estabilidade, é também desejado que a convergência dos estados para zero seja feita com critérios de velocidade e oscilação específicos do projeto. Isso é equivalente a satisfazer as condições de overshoot, tempo de acomodação e dominância que você aprendeu a fazer no projeto com função de trsferência.

Vamos supor que essas condições de projeto estejam definidas em termos de pólos de malha fechada bem selecionados e que esses pólos determinam um polinômio de malha fechada $\alpha (s)$.

Sendo assim, o problema de regulação é enunciado como: *achar a matriz $\mathbf{K}$,  tal que estejam no semi-plano esquerdo todas as raizes da equação:*
$$
\begin{align}
    \det (s\mathbf{I-F_m})= \det (s\mathbf{I-F+BK})=\alpha (s)
\end{align}
$$


## Exemplo

<p align="center">
<img src="Ex7.15.svg" width="70%">
</p>

Como os parâmetros do problema não são numéricos, precisamos resolver usando matemática simbólica.

In [15]:
import numpy as np
import sympy as sp

# Define os símbolos
w0 = sp.symbols('w0',real=True)
s = sp.symbols('s')
k1,k2 = sp.symbols(['k1','k2'],real=True)

# Define as matrizes do sistema
F = sp.Matrix([[0,1],[-w0**2,0]])
G = sp.Matrix([[0],[1]])

# Define o vetor de ganhos
K = sp.Matrix([k1,k2]).T      # Transpor para virar vetor linha

print('Calcule primeiro GK')
G*K


Calcule primeiro GK


Matrix([
[ 0,  0],
[k1, k2]])

In [16]:
print('Agora faça F-GK')
# Calcula a matriz de malha fechada
Fm = F-G*K
Fm

Agora faça F-GK


Matrix([
[          0,   1],
[-k1 - w0**2, -k2]])

In [22]:
print('Calcule sI (lembre-se que o sistema é de 2a ordem!)')
s*sp.eye(2)

Calcule sI (lembre-se que o sistema é de 2a ordem!)


Matrix([
[s, 0],
[0, s]])

In [18]:
print("Agora faça sI-Fm")
s*sp.eye(2)-Fm

Agora faça sI-Fm


Matrix([
[         s,     -1],
[k1 + w0**2, k2 + s]])

In [31]:
print('Calcule o determinante. Esse é o polinomio de malha fechada de projeto')
p=sp.det(s*sp.eye(2)-Fm)
p

Calcule o determinante. Esse é o polinomio de malha fechada de projeto


k1 + k2*s + s**2 + w0**2

In [29]:
print('Deseja-se que os dois polos de malha fechada sejam em -2w0')
print('O polinômio de malha fechada desejado com dois polos nessa posição é :')
alpha = sp.expand((s+2*w0)**2)
alpha


Deseja-se que os dois polos de malha fechada sejam em -2w0
O polinômio de malha fechada desejado com dois polos nessa posição é :


s**2 + 4*s*w0 + 4*w0**2

In [40]:
print('Agora montamos equações igualando os coeficientes')
equacoes=[sp.Eq(alpha.coeff(s,n),p.coeff(s,n)) for n in (0,1)]
print('Primeira equação (coeficientes em s)')
equacoes[0]

Agora montamos equações igualando os coeficientes
Primeira equação (coeficientes em s)


Eq(4*w0**2, k1 + w0**2)

In [42]:
print('Segunda equação')
equacoes[1]

Segunda equação


Eq(4*w0, k2)

In [50]:
print('Basta resolver agora para k1 e k2')
sol=sp.solve(equacoes,[k1,k2])
sp.Eq(k1,sol[k1])

Basta resolver agora para k1 e k2


Eq(k1, 3*w0**2)

In [51]:
sp.Eq(k2,sol[k2])

Eq(k2, 4*w0)

O livro mostra uma simulação da resposta desse sistema para $\omega_0=1$. Repare que os gráficos mostram, em função do tempo, os dois estados. 

<p align="center">
<img src="Fig7.13.svg" width="90%">
</p>

Note que nenhum dos gráficos oscila. Isso porque as características dos pólos que foram alocados são válidas para todos os estados e a saída do sistema.

## Exercício

Ache a forma de controlador do sistema
$$
\begin{align}
    G(s) = \frac{20}{(s+1)(s+2)}
\end{align}
$$

Depois, projete um controlador de realimentação de estados que posicione os pólos em $-3\pm j5$